<a href="https://colab.research.google.com/github/justalge/another_python_tutorial/blob/main/week5/Lecture_9_descriptors_inheritance_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descriptors

Descriptors were introduced to Python in version 2.2. That time the "What's New in Python2.2" mentioned: "The one big idea underlying the new class model is that an API for describing the attributes of an object using descriptors has been formalized. Descriptors specify the value of an attribute, stating whether it’s a method or a field. With the descriptor API, static methods and class methods become possible, as well as more exotic constructs."

**A descriptor is an object attribute with "binding behavior", one whose attribute access has been overridden by methods in the descriptor protocol. Those methods are \_\_get__(),  \_\_set__(), and \_\_delete__()**

If any of those methods are defined for an object, it is said to be a descriptor.

Let a closer look at what is happening. Assuming we have an object obj: What happens if we try to access an attribute (or property) ap? "Accesssing" the attribute means to "get" the value, so the attribute is used for example in a print function or inside of an expression. Both the obj and the class belong to type(obj) contain a dictionary attribute __dict__. This situation is viuslaized in the following diagram:

![](https://www.python-course.eu/images/lookup_chain_properties_700w.webp)

`obj.ap` has a lookup chain starting with `obj.__dict__['ap']`, i.e. checks if obj.ap is a key of the dictionary `obj.__dict__['ap']`

![](https://www.python-course.eu/images/lookup_chain_properties2_700w.webp)

If ap is not a key of `obj.__dict__`, it will try to lookup `type(obj).__dict__['ap']`

![](https://www.python-course.eu/images/lookup_chain_properties3_700w.webp)

If obj is not contained in this dictionary either, it will continue checking through the base classes of type(ap) excluding metaclasses.

We demonstrate this in an example:

In [ ]:
class A:
    
    ca_A = "class attribute of A"
    
    def __init__(self):
        self.ia_A = "instance attribute of A instance"
        
class B(A):
 
    ca_B = "class attribute of B"
    
    def __init__(self):
        super().__init__()
        self.ia_B = "instance attribute of A instance"

x = B()

print(x.ia_B)
print(x.ca_B)
print(x.ia_A)
print(x.ca_A)

instance attribute of A instance
class attribute of B
instance attribute of A instance
class attribute of A


If we call `print(x.non_existing)` we get the following exception:

In [ ]:
print(x.non_existing)

AttributeError: ignored

If the looked-up value is an object defining one of the descriptor methods, then Python may override the default behavior and invoke the descriptor method instead. Where this occurs in the precedence chain depends on which descriptor methods were defined.

Descriptors provide a powerful, general purpose protocol, which is the underlying mechanism behind properties, methods, static methods, class methods, and super(). Descriptors were needed to implement the so-called new style classes introduced in version 2.2. The "new style classes" are the default nowadays

#### Descriptor protocol

The general descriptor protocol consists of three methods:

* `descr.__get__(self, obj, type=None) -> value`
* `descr.__set__(self, obj, value) -> None`
* `descr.__delete__(self, obj) -> None`

If you define one or more of these methods, you will create a descriptor. We distinguish between data descriptors and non-data descriptors:

* **non-data descriptor** If we define only the \_\_get__() method, we create a non-data descriptor, which are mostly used for methods
* **data descriptor** If an object defines \_\_set__() or \_\_delete__(), it is considered a data descriptor. To make a read-only data descriptor, define both \_\_get__() and \_\_set__() with the \_\_set__() raising an AttributeError when called. Defining the __set__() method with an exception raising placeholder is enough to make it a data descriptor

We finally come now to our simple descriptor example in the following code:

In [ ]:
class SimpleDescriptor(object):
    """
    A simple data descriptor that can set and return values
    """

    def __init__(self, initval=None):
        print("__init__ of SimpleDecorator called with initval: ", initval)
        self.__set__(self, initval)

    def __get__(self, instance, owner):
        print('Instance:', instance, 'Owner', owner)
        print('Getting (Retrieving) self.val: ', self.val)
        return self.val

    def __set__(self, instance, value):
        print('Setting self.val to ', value)
        self.val = value

class MyClass(object):
    
    x = SimpleDescriptor("green")

__init__ of SimpleDecorator called with initval:  green
Setting self.val to  green


In [ ]:
m = MyClass()

In [ ]:
print(m.x)

Instance: <__main__.MyClass object at 0x7fe368477e90> Owner <class '__main__.MyClass'>
Getting (Retrieving) self.val:  green
green


In [ ]:
m

In [ ]:
m.x = "yellow"

Setting self.val to  yellow


In [ ]:
print(m.x)

Instance: <__main__.MyClass object at 0x7fe368477e90> Owner <class '__main__.MyClass'>
Getting (Retrieving) self.val:  yellow
yellow


The third parameter owner of \_\_get__ is always the owner class and provides users with an option to do something with the class that was used to call the descriptor. Usually, i.e. if the descriptor is called through an object obj, the object type can be deduced by calling type(obj). The situation is different, if the descriptor is invoked through a class. In this case it is None and it wouldn't be possible to access the class unless the third argument is given. The second parameter instance is the instance that the attribute was accessed through, or None when the attribute is accessed through the owner.

Let's have a look at the \_\_dict__ dictionaries of both the instances and the class:

In [ ]:
print(m.__dict__)

{}


In [ ]:
print(MyClass.__dict__)

{'__module__': '__main__', 'x': <__main__.SimpleDescriptor object at 0x7fe36845a290>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}


In [ ]:
print(SimpleDescriptor.__dict__)

{'__module__': '__main__', '__doc__': '\n    A simple data descriptor that can set and return values\n    ', '__init__': <function SimpleDescriptor.__init__ at 0x7fe36840c710>, '__get__': <function SimpleDescriptor.__get__ at 0x7fe36840c8c0>, '__set__': <function SimpleDescriptor.__set__ at 0x7fe36840c200>, '__dict__': <attribute '__dict__' of 'SimpleDescriptor' objects>, '__weakref__': <attribute '__weakref__' of 'SimpleDescriptor' objects>}


The methods \_\_get__(),  \_\_set__(), and \_\_delete__() will only apply if an instance of the class containing the method (a so-called descriptor class) appears in an owner class (the descriptor must be in either the owner’s class dictionary or in the class dictionary for one of its parents). In the examples above, the attribute 'x' is in the owner \_\_dict__ of the owner class MyClass

For example, obj.d looks up d in the dictionary \_\_dict__ of obj. If d defines the method \_\_get__(), then d.\_\_get__(obj) is invoked according to the precedence rules listed below.

It makes a difference if obj is an object or a class:

* For objects, the method to control the invocation is in object.\_\_getattribute__() which transforms b.x into the call type(b).\_\_dict__['x'].\_\_get__(b, type(b)). The implementation works through a precedence chain that gives data descriptors priority over instance variables, instance variables priority over non-data descriptors, and assigns lowest priority to \_\_getattr__() if provided
* For classes, the corresponing method is in the type class, i.e. type.\_\_getattribute__() which transforms B.x into B.\_\_dict__['x'].\_\_get__(None, B)

\_\_getattribute__ is not implemented in Python but in C. The following Python code is a simulation of the logic in Python. We can see that the descriptors are called by the \_\_getattribute__ implementations

In [ ]:
def __getattribute__(self, key):
    "Emulate type_getattro() in Objects/typeobject.c"
    v = type.__getattribute__(self, key)
    if hasattr(v, '__get__'):
        return v.__get__(None, self)
    return v

m.__getattribute__("x")

Instance: <__main__.MyClass object at 0x7fe368477e90> Owner <class '__main__.MyClass'>
Getting (Retrieving) self.val:  yellow


'yellow'

#### Understanding the difference between \_\_getattr__ and \_\_getattribute__ and how to avoid infinite recursion

##### Use of \_\_getattr__

Python will call this method whenever you request an attribute that hasn't already been defined, so you can define what to do with it

A classic use case:

In [ ]:
class A:
    def __getattr__(self, name):
       return 'Attribute doessn\'t exist'
a = A()
a.x = 5
# Now a.somekey will give th string above:
print(a.somekey)
a.x

Attribute doessn't exist


5

##### Use of \_\_getattribute__

If you need to catch every attribute regardless whether it exists or not, use \_\_getattribute__ instead. The difference is that \_\_getattr__ only gets called for attributes that don't actually exist. If you set an attribute directly, referencing that attribute will retrieve it without calling \_\_getattr__

\_\_getattribute__ is called all the times:

In [ ]:
class Foo(object):
    def __init__(self, a):
        self.a = 1

    def __getattribute__(self, attr):
        try:
            return self.__dict__[attr]
        except KeyError:
            return 'default'
f = Foo(1)
f.a

RecursionError: ignored

This will cause infinite recursion. The culprit here is the line return `self.__dict__[attr]`. Let's pretend (It's close enough to the truth) that all attributes are stored in self.\_\_dict__ and available by their name. The line

`f.a`

attempts to access the a attribute of f. This calls `f.__getattribute__('a')` \_\_dict__ is an attribute of self == f and so python calls `f.__getattribute__('__dict__')` which again tries to access the attribute \_\_dict__. This is infinite recursion. The 'correct' way to write the above class using \_\_getattribute__ is:

In [ ]:
class Foo(object):
    def __init__(self, a):
        self.a = 1

    def __getattribute__(self, attr):
        return super(Foo, self).__getattribute__(attr)

f = Foo(1)
f.a

1

`super(Foo, self).__getattribute__(attr)` binds the \_\_getattribute__ method of the 'nearest' superclass (formally, the next class in the class's Method Resolution Order, or MRO) to the current object self and then calls it and lets that do the work. More [here](https://newbedev.com/understanding-the-difference-between-getattr-and-getattribute)

The object returned by super() also has a custom \_\_getattribute__() method for invoking descriptors. The attribute lookup super(B, obj).m searches obj.\_\_class__.\_\_mro__ for the base class A immediately following B and then returns A.\_\_dict__['m'].\_\_get__(obj, B). If not a descriptor, m is returned unchanged. If not in the dictionary, m reverts to a search using object.\_\_getattribute__()

The details above show that the mechanism for descriptors is embedded in the \_\_getattribute__() methods for object, type, and super(). Classes inherit this machinery when they derive from object or if they have a meta-class providing similar functionality. This means also that one can turn-off automatic descriptor calls by overriding \_\_getattribute__()

#### Examples:

In [ ]:
from weakref import WeakKeyDictionary
 
class Voter:
    
    required_age = 18 # in Germany
    
    def __init__(self):
        self.age = WeakKeyDictionary()
 
    def __get__(self, instance_obj, objtype):
        return self.age.get(instance_obj)
 
    def __set__(self, instance, new_age):
        if new_age < Voter.required_age:
            msg = '{name} is not old enough to vote in Germany'
            raise Exception(msg.format(name=instance.name))
        self.age[instance] = new_age
        print('{name} can vote in Germany'.format(
            name=instance.name))
 
    def __delete__(self, instance):
        del self.age[instance]
 
 
class Person:
    voter_age = Voter()
 
    def __init__(self, name, age):
        self.name = name
        self.voter_age = age
 
 
p1 = Person('Ben', 23)
p2 = Person('Emilia', 22)

p2.voter_age

Ben can vote in Germany
Emilia can vote in Germany


22

#### [A pure sample implementation of a property() class](https://docs.python.org/3/howto/descriptor.html#properties)

[How does the @property decorator work in Python](https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work-in-python)
Let's make our Property implementation a little bit more talkative with some print functions to see what is going on:



In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        print("\n__init__ of Property called with:")
        print("fget=" + str(fget) + " fset=" + str(fset) + \
              " fdel=" + str(fdel) + " doc=" + str(doc))
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        print("\nProperty.__get__ has been called!")
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        print("\nProperty.__set__ has been called!")
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        print("\nProperty.__delete__ has been called!")
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        print("\nProperty.getter has been called!")
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        print("\nProperty.setter has been called!")
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        print("\nProperty.deleter has been called!")
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
    
class A:
    
    def __init__(self, prop):
        self.prop = prop
    
    @Property
    def prop(self):
        """ This will be the doc string of the property """
        print("The Property 'prop' will be returned now:")
        return self.__prop
    
    @prop.setter
    def prop(self, prop):
        print("prop will be set")
        self.__prop = prop
        
    def prop2_getter(self):
        return self.__prop2
    
    def prop2_setter(self, prop2):
        self.__prop2 = prop2
        
    prop2 = Property(prop2_getter, prop2_setter)
        
print("Initializing the Property 'prop' with the value 'Python'")
x = A("Python")
print("The value is: ", x.prop)
print("Reassigning the Property 'prop' to 'Python descriptors'")
x.prop = "Python descriptors"
print("The value is: ", x.prop)

print(A.prop.getter(x))

def new_prop_setter(self, prop):
    if prop=="foo":
        self.__prop = "foobar"
    else:
        self.__prop = prop
        
A.prop.setter


__init__ of Property called with:
fget=<function A.prop at 0x7fe368418830> fset=None fdel=None doc=None

Property.setter has been called!

__init__ of Property called with:
fget=<function A.prop at 0x7fe368418830> fset=<function A.prop at 0x7fe368418320> fdel=None doc= This will be the doc string of the property 

__init__ of Property called with:
fget=<function A.prop2_getter at 0x7fe368439830> fset=<function A.prop2_setter at 0x7fe3684398c0> fdel=None doc=None
Initializing the Property 'prop' with the value 'Python'

Property.__set__ has been called!
prop will be set

Property.__get__ has been called!
The Property 'prop' will be returned now:
The value is:  Python
Reassigning the Property 'prop' to 'Python descriptors'

Property.__set__ has been called!
prop will be set

Property.__get__ has been called!
The Property 'prop' will be returned now:
The value is:  Python descriptors

Property.__get__ has been called!

Property.getter has been called!

__init__ of Property called with:
f

<bound method Property.setter of <__main__.Property object at 0x7fe36bf1ae90>>

A simple class using our Property implementation:

In [ ]:
class Robot:
    def __init__(self, name="Marvin", city="Freiburg"):
        self.name = name
        self.city = city
          
    @Property
    def name(self):
        return self.__name
    
    @name.setter
    def name(self, name):
        if name == "hello":
            self.__name = "hi"
        else:
            self.__name = name



__init__ of Property called with:
fget=<function Robot.name at 0x7fe368439d40> fset=None fdel=None doc=None

Property.setter has been called!

__init__ of Property called with:
fget=<function Robot.name at 0x7fe368439d40> fset=<function Robot.name at 0x7fe368439a70> fdel=None doc=None


Using our class Robot:

In [ ]:
x = Robot("Marvin")
print(x.name)
x.name = "Eddie"
print(x.name)


Property.__set__ has been called!

Property.__get__ has been called!
Marvin

Property.__set__ has been called!

Property.__get__ has been called!
Eddie


#### Is it possible to automatically create descriptors at runtime?

Yes

In [ ]:
class DynPropertyClass(object):

    def add_property(self, attribute):
        """ add a property to the class """        

        def get_attribute(self):
            """ The value for attribute 'attribute' will be retrieved """
            return getattr(self, "_" + type(x).__name__ + "__" + attribute)
        
        def set_attribute(self, value):
            """ The value for attribute 'attribute' will be retrieved """
            #setter = lambda self, value: self.setProperty(attribute, value)
            setattr(self, "_" + type(x).__name__ + "__" + attribute, value)

        #construct property attribute and add it to the class
        setattr(type(self), attribute, property(fget=get_attribute, 
                                                fset=set_attribute, 
                                                doc="Auto‑generated method"))


    
x = DynPropertyClass()
x.add_property('name')
x.add_property('city')
x.name = "Henry"
x.name
x.city = "Hamburg"
print(x.name, x.city)
print(x.__dict__)

Henry Hamburg
{'_DynPropertyClass__name': 'Henry', '_DynPropertyClass__city': 'Hamburg'}


## Inheritance

![](https://www.python-course.eu/images/dna2_800w.webp)

No object-oriented programming language would be worthy to look at or use, if it didn't support inheritance. Inheritance was invented in 1969 for Simula. Python not only supports inheritance but multiple inheritance as well. Generally speaking, inheritance is the mechanism of deriving new classes from existing ones. By doing this, we get a hierarchy of classes. In most class-based object-oriented languages, an object created through inheritance (a "child object") acquires all, - though there are exceptions in some programming languages, - of the properties and behaviors of the parent object.

Inheritance allows programmers to create classes that are built upon existing classes, and this enables a class created through inheritance to inherit the attributes and methods of the parent class. This means that inheritance supports code reusability. The methods or generally speaking the software inherited by a subclass is considered to be reused in the subclass. The relationships of objects or classes through inheritance give rise to a directed graph.

The class from which a class inherits is called the parent or superclass. A class which inherits from a superclass is called a subclass, also called heir class or child class. Superclasses are sometimes called ancestors as well. There exists a hierarchical relationship between classes. It's similar to relationships or categorizations that we know from real life. Think about vehicles, for example. Bikes, cars, buses and trucks are vehicles. Pick-ups, vans, sports cars, convertibles and estate cars are all cars and by being cars they are vehicles as well. We could implement a vehicle class in Python, which might have methods like accelerate and brake. Cars, Buses and Trucks and Bikes can be implemented as subclasses which will inherit these methods from vehicle.

![](https://www.python-course.eu/images/vehicles_classification_800w.webp)

#### Syntax of Inheritance in Python

The syntax for a subclass definition looks like this:


```python
class DerivedClassName(BaseClassName):
    pass
```


Instead of the pass statement, there will be methods and attributes like in all other classes. The name BaseClassName must be defined in a scope containing the derived class definition.

Now we are ready for a simple inheritance example with Python code:

In [73]:
class Robot:
    
    def __init__(self, name):
        self.name = name
        
    def say_hi(self):
        print("Hi, I am " + self.name)
        
class PhysicianRobot(Robot):
    pass

x = Robot("Marvin")
y = PhysicianRobot("James")

print(x, type(x))
print(y, type(y))

y.say_hi()

<__main__.Robot object at 0x7fe368433150> <class '__main__.Robot'>
<__main__.PhysicianRobot object at 0x7fe36842a110> <class '__main__.PhysicianRobot'>
Hi, I am James


![](https://www.python-course.eu/images/marvin_and_james_300w.webp)

If you look at the code of our PhysicianRobot class, you can see that we haven't defined any attributes or methods in this class. As the class PhysicianRobot is a subclass of Robot, it inherits, in this case, both the method \_\_init__ and say_hi. Inheriting these methods means that we can use them as if they were defined in the PhysicianRobot class. When we create an instance of PhysicianRobot, the \_\_init__ function will also create a name attribute. We can apply the say_hi method to the PhysisicianRobot object y, as we can see in the output from the code above

#### Difference between type and isinstance

You should also pay attention to the following facts, which we pointed out in other sections of our Python tutorial as well. People frequently ask where the difference between checking the type via the type function or the function isinstance is lies. The difference can be seen in the following code. We see that isinstance returns True if we compare an object either with the class it belongs to or with the superclass. Whereas the equality operator only returns True, if we compare an object with its own class:

In [74]:
x = Robot("Marvin")
y = PhysicianRobot("James")

print(isinstance(x, Robot), isinstance(y, Robot))
print(isinstance(x, PhysicianRobot))
print(isinstance(y, PhysicianRobot))

print(type(y) == Robot, type(y) == PhysicianRobot)

True True
False
True
False True


This is even true for arbitrary ancestors of the class in the inheritance line:

In [75]:
class A:
    pass

class B(A):
    pass

class C(B):
    pass

x = C()
print(isinstance(x, A))

True


Now it should be clear, why PEP 8, the official Style Guide for Python code, says: "Object type comparisons should always use isinstance() instead of comparing types directly."

#### Overriding

Let us get back to our new PhysicianRobot class. Imagine now that an instance of a PhysicianRobot should say hi in a different way. In this case, we have to redefine the method say_hi inside of the subclass PhysicianRobot:

In [76]:
class Robot:
    
    def __init__(self, name):
        self.name = name
        
    def say_hi(self):
        print("Hi, I am " + self.name)
        
class PhysicianRobot(Robot):

    def say_hi(self):
        print("Everything will be okay! ") 
        print(self.name + " takes care of you!")

y = PhysicianRobot("James")
y.say_hi()

Everything will be okay! 
James takes care of you!


What we have done in the previous example is called overriding. A method of a parent class gets overridden by simply defining a method with the same name in the child class.

If a method is overridden in a class, the original method can still be accessed, but we have to do it by calling the method directly with the class name, i.e. Robot.say_hi(y). We demonstrate this in the following code:

In [77]:
y = PhysicianRobot("Doc James")
y.say_hi()
print("... and now the 'traditional' robot way of saying hi :-)")
Robot.say_hi(y)

Everything will be okay! 
Doc James takes care of you!
... and now the 'traditional' robot way of saying hi :-)
Hi, I am Doc James



We have seen that an inherited class can inherit and override methods from the superclass. Besides this a subclass often needs additional methods with additional functionalities, which do not exist in the superclass. An instance of the PhysicianRobot class will need for example the method heal so that the physician can do a proper job. We will also add an attribute health_level to the Robot class, which can take a value between 0 and 1. The robots will 'come to live' with a random value between 0 and 1. If the health_level of a Robot is below 0.8, it will need a doctor. We write a method needs_a_doctor which returns True if the value is below 0.8 and False otherwise. The 'healing' in the heal method is done by setting the health_level to a random value between the old health_level and 1. This value is calculated by the uniform function of the random module:

In [78]:
import random

class Robot:
    
    def __init__(self, name):
        self.name = name
        self.health_level = random.random() 
        
    def say_hi(self):
        print("Hi, I am " + self.name)
        
    def needs_a_doctor(self):
        if self.health_level < 0.8:
            return True
        else:
            return False
        
class PhysicianRobot(Robot):

    def say_hi(self):
        print("Everything will be okay! ") 
        print(self.name + " takes care of you!")


    def heal(self, robo):
        robo.health_level = random.uniform(robo.health_level, 1)
        print(robo.name + " has been healed by " + self.name + "!")

doc = PhysicianRobot("Dr. Frankenstein")        

rob_list = []
for i in range(5):
    x = Robot("Marvin" + str(i))
    if x.needs_a_doctor():
        print("health_level of " + x.name + " before healing: ", x.health_level)
        doc.heal(x)
        print("health_level of " + x.name + " after healing: ", x.health_level)
    rob_list.append((x.name, x.health_level))
    
print(rob_list)

health_level of Marvin0 before healing:  0.020428390216115866
Marvin0 has been healed by Dr. Frankenstein!
health_level of Marvin0 after healing:  0.11868232649086248
health_level of Marvin1 before healing:  0.44813828827448043
Marvin1 has been healed by Dr. Frankenstein!
health_level of Marvin1 after healing:  0.9277522523483387
health_level of Marvin2 before healing:  0.2715805478123987
Marvin2 has been healed by Dr. Frankenstein!
health_level of Marvin2 after healing:  0.8555385329361471
health_level of Marvin3 before healing:  0.34021522998368825
Marvin3 has been healed by Dr. Frankenstein!
health_level of Marvin3 after healing:  0.909210886755749
[('Marvin0', 0.11868232649086248), ('Marvin1', 0.9277522523483387), ('Marvin2', 0.8555385329361471), ('Marvin3', 0.909210886755749), ('Marvin4', 0.9682103489338649)]



When we override a method, we sometimes want to reuse the method of the parent class and at some new stuff. To demonstrate this, we will write a new version of the PhysicianRobot. say_hi should return the text from the Robot class version plus the text " and I am a physician!":

In [80]:
class PhysicianRobot(Robot):

    def say_hi(self):
        Robot.say_hi(self)
        print("and I am a physician!")

        
doc = PhysicianRobot("Dr. Frankenstein")
doc.say_hi()

Hi, I am Dr. Frankenstein
and I am a physician!



We don't want to write redundant code and therefore we called Robot.say_hi(self). We could also use the super function:

In [81]:
class PhysicianRobot(Robot):

    def say_hi(self):
        super().say_hi()
        print("and I am a physician!")

        
doc = PhysicianRobot("Dr. Frankenstein")      
doc.say_hi()

Hi, I am Dr. Frankenstein
and I am a physician!


`super` is not realls necessary in this case. One could argue that it makes the code more maintainable, because we could change the name of the parent class, but this is seldom done anyway in existing classes. The real benefit of super shows when we use it with multiple inheritance

#### Distinction between Overwriting, Overloading and Overriding

##### Overwriting

If we overwrite a function, the original function will be gone. The function will be redefined. This process has nothing to do with object orientation or inheritance:

In [82]:
def f(x):
    return x + 42

print(f(3))
# f will be overwritten (or redefined) in the following:

def f(x):
    return x + 43
print(f(3))

45
46


##### Overloading

This subchapter will be only interesting for C++ and Java programmers who want to know how overloading can be accomplished in Python. Those who do not know about overloading will not miss it!

In the context of object-oriented programming, you might have heard about "overloading" as well. Even though "overloading" is not directly connected to OOP. Overloading is the ability to define a function with the same name multiple times. The definitions are different concerning the number of parameters and types of the parameters. It's the ability of one function to perform different tasks, depending on the number of parameters or the types of the parameters. We cannot overload functions like this in Python, but it is not necessary either.

This course is, however, not about C++ and we have so far avoided using any C++ code. We want to make an exception now, so that you can see, how overloading works in C++

```
#include 
#include 

using namespace std;

int successor(int number) {
    return number + 1;
}

double successor(double number) {
    return number + 1;
}

int main() {

    cout << successor(10) << endl;
    cout << successor(10.3) << endl;

    return 0;
}
```

We defined the successor function twice: One time for int and the other time with float as a Parameter. In Python the function can be defined like this, as you will know for sure:

In [83]:
def successor(x):
    return x + 1

As x is only a reference to an object, the Python function successor can be called with every object, even though it will create exceptions with many types. But it will work with int and float values!

Having a function with a different number of parameters is another way of function overloading. The following C++ program shows such an example. The function f can be called with either one or two integer arguments:

```
 #include 
using namespace std;


int f(int n);
int f(int n, int m);

int main() {

    cout << "f(3): " << f(3) << endl;
    cout << "f(3, 4): " << f(3, 4) << endl;
    return 0;
}

int f(int n) {
    return n + 42;
}
int f(int n, int m) {
    return n + m + 42; 
}
```


This doesn't work in Python, as we can see in the following example. The second definition of f with two parameters redefines or overrides the first definition with one argument. Overriding means that the first definition is not available anymore:

In [84]:

def f(n):
    return n + 42
 
def f(n,m):
    return n + m + 42

print(f(3, 4))

49



If you call f with only one parameter, you will raise an exception:

In [85]:
f(3)

TypeError: ignored


Yet, it is possible to simulate the overloading behaviour of C++ in Python in this case with a default parameter:

In [86]:
def f(n, m=None):
    if m:
        return n + m +42
    else:
        return n + 42

print(f(3), f(1, 3))

45 46


The * operator can be used as a more general approach for a family of functions with 1, 2, 3, or even more parameters:

In [87]:
def f(*x):
    if len(x) == 1:
        return x[0] + 42
    elif len(x) == 2:
        return x[0] - x[1] + 5
    else:
        return 2 * x[0] + x[1] + 42

print(f(3), f(1, 2), f(3, 2, 1))

45 4 50


## Multiple Inheritance

![](https://www.python-course.eu/images/diamond_800w.webp)

In the previous chapter of our tutorial, we have covered inheritance, or more specific "single inheritance". As we have seen, a class inherits in this case from one class. Multiple inheritance on the other hand is a feature in which a class can inherit attributes and methods from more than one parent class. The critics point out that multiple inheritance comes along with a high level of complexity and ambiguity in situations such as the diamond problem. We will address this problem later in this chapter.

The widespread prejudice that multiple inheritance is something "dangerous" or "bad" is mostly nourished by programming languages with poorly implemented multiple inheritance mechanisms and above all by improper usage of it. Java doesn't even support multiple inheritance, while C++ supports it. Python has a sophisticated and well-designed approach to multiple inheritance.

A class definition, where a child class SubClassName inherits from the parent classes BaseClass1, BaseClass2, BaseClass3, and so on, looks like this:

```python
class SubclassName(BaseClass1, BaseClass2, BaseClass3, ...):
    pass
```

It's clear that all the superclasses BaseClass1, BaseClass2, BaseClass3, ... can inherit from other superclasses as well. What we get is an inheritance tree:

![](https://www.python-course.eu/images/multiple_inheritance_700w.webp)

#### Example: CalendarClock

We want to introduce the principles of multiple inheritance with an example. For this purpose, we will implement to independent classes: a "Clock" and a "Calendar" class. After this, we will introduce a class "CalendarClock", which is, as the name implies, a combination of "Clock" and "Calendar". CalendarClock inherits both from "Clock" and "Calendar":

![](https://www.python-course.eu/images/clock_calendar_800w.webp)

The class Clock simulates the tick-tack of a clock. An instance of this class contains the time, which is stored in the attributes self.hours, self.minutes and self.seconds. Principally, we could have written the /_/_init__ method and the set method like this:

```python
def __init__(self,hours=0, minutes=0, seconds=0):
        self._hours = hours
        self.__minutes = minutes
        self.__seconds = seconds

    def set(self,hours, minutes, seconds=0):
        self._hours = hours
        self.__minutes = minutes
        self.__seconds = seconds
```

We decided against this implementation, because we added additional code for checking the plausibility of the time data into the set method. We call the set method from the \_\_init__ method as well, because we want to circumvent redundant code. The complete Clock class:

In [88]:
""" 
The class Clock is used to simulate a clock.
"""

class Clock(object):

    def __init__(self, hours, minutes, seconds):
        """
        The paramaters hours, minutes and seconds have to be 
        integers and must satisfy the following equations:
        0 <= h < 24
        0 <= m < 60
        0 <= s < 60
        """

        self.set_Clock(hours, minutes, seconds)

    def set_Clock(self, hours, minutes, seconds):
        """
        The parameters hours, minutes and seconds have to be 
        integers and must satisfy the following equations:
        0 <= h < 24
        0 <= m < 60
        0 <= s < 60
        """

        if type(hours) == int and 0 <= hours and hours < 24:
            self._hours = hours
        else:
            raise TypeError("Hours have to be integers between 0 and 23!")
        if type(minutes) == int and 0 <= minutes and minutes < 60:
            self.__minutes = minutes 
        else:
            raise TypeError("Minutes have to be integers between 0 and 59!")
        if type(seconds) == int and 0 <= seconds and seconds < 60:
            self.__seconds = seconds
        else:
            raise TypeError("Seconds have to be integers between 0 and 59!")

    def __str__(self):
        return "{0:02d}:{1:02d}:{2:02d}".format(self._hours,
                                                self.__minutes,
                                                self.__seconds)

    def tick(self):
        """
        This method lets the clock "tick", this means that the 
        internal time will be advanced by one second.

        Examples:
        >>> x = Clock(12,59,59)
        >>> print(x)
        12:59:59
        >>> x.tick()
        >>> print(x)
        13:00:00
        >>> x.tick()
        >>> print(x)
        13:00:01
        """

        if self.__seconds == 59:
            self.__seconds = 0
            if self.__minutes == 59:
                self.__minutes = 0
                if self._hours == 23:
                    self._hours = 0
                else:
                    self._hours += 1
            else:
                self.__minutes += 1
        else:
            self.__seconds += 1


if __name__ == "__main__":
    x = Clock(23,59,59)
    print(x)
    x.tick()
    print(x)
    y = str(x)
    print(type(y))

23:59:59
00:00:00
<class 'str'>


Let's check our exception handling by inputting floats and strings as input. We also check, what happens, if we exceed the limits of the expected values:

In [89]:
x = Clock(7.7, 45, 17)

TypeError: ignored

We will now create a class "Calendar", which has lots of similarities to the previously defined Clock class. Instead of "tick" we have an "advance" method, which advances the date by one day, whenever it is called. Adding a day to a date is quite tricky. We have to check, if the date is the last day in a month and the number of days in the months vary. As if this isn't bad enough, we have February and the leap year problem.

The rules for calculating a leap year are the following:

* If a year is divisible by 400, it is a leap year
* If a year is not divisible by 400 but by 100, it is not a leap year
* A year number which is divisible by 4 but not by 100, it is a leap year
* All other year numbers are common years, i.e. no leap years

As a little useful gimmick, we added a possibility to output a date either in British or in American (Canadian) style:

In [90]:
""" 
The class Calendar implements a calendar.   
"""

class Calendar(object):

    months = (31,28,31,30,31,30,31,31,30,31,30,31)
    date_style = "British"

    @staticmethod
    def leapyear(year):
        """ 
        The method leapyear returns True if the parameter year
        is a leap year, False otherwise
        """
        if not year % 4 == 0:
            return False
        elif not year % 100 == 0:
            return True
        elif not year % 400 == 0:
            return False
        else:
            return True


    def __init__(self, d, m, y):
        """
        d, m, y have to be integer values and year has to be 
        a four digit year number
        """

        self.set_Calendar(d,m,y)


    def set_Calendar(self, d, m, y):
        """
        d, m, y have to be integer values and year has to be 
        a four digit year number
        """

        if type(d) == int and type(m) == int and type(y) == int:
            self.__days = d
            self.__months = m
            self.__years = y
        else:
            raise TypeError("d, m, y have to be integers!")


    def __str__(self):
        if Calendar.date_style == "British":
            return "{0:02d}/{1:02d}/{2:4d}".format(self.__days,
                                                   self.__months,
                                                   self.__years)
        else: 
            # assuming American style
            return "{0:02d}/{1:02d}/{2:4d}".format(self.__months,
                                                   self.__days,
                                                   self.__years)



    def advance(self):
        """
        This method advances to the next date.
        """

        max_days = Calendar.months[self.__months-1]
        if self.__months == 2 and Calendar.leapyear(self.__years):
            max_days += 1
        if self.__days == max_days:
            self.__days= 1
            if self.__months == 12:
                self.__months = 1
                self.__years += 1
            else:
                self.__months += 1
        else:
            self.__days += 1


if __name__ == "__main__":
    x = Calendar(31,12,2012)
    print(x, end=" ")
    x.advance()
    print("after applying advance: ", x)
    print("2012 was a leapyear:")
    x = Calendar(28,2,2012)
    print(x, end=" ")
    x.advance()
    print("after applying advance: ", x)
    x = Calendar(28,2,2013)
    print(x, end=" ")
    x.advance()
    print("after applying advance: ", x)
    print("1900 no leapyear: number divisible by 100 but not by 400: ")
    x = Calendar(28,2,1900)
    print(x, end=" ")
    x.advance()
    print("after applying advance: ", x)
    print("2000 was a leapyear, because number divisibe by 400: ")
    x = Calendar(28,2,2000)
    print(x, end=" ")
    x.advance()
    print("after applying advance: ", x)
    print("Switching to American date style: ")
    Calendar.date_style = "American"
    print("after applying advance: ", x)

31/12/2012 after applying advance:  01/01/2013
2012 was a leapyear:
28/02/2012 after applying advance:  29/02/2012
28/02/2013 after applying advance:  01/03/2013
1900 no leapyear: number divisible by 100 but not by 400: 
28/02/1900 after applying advance:  01/03/1900
2000 was a leapyear, because number divisibe by 400: 
28/02/2000 after applying advance:  29/02/2000
Switching to American date style: 
after applying advance:  02/29/2000


At last, we will introduce our multiple inheritance example. We are now capable of implementing the originally intended class CalendarClock, which will inherit from both Clock and Calendar. The method "tick" of Clock will have to be overridden. However, the new tick method of CalendarClock has to call the tick method of Clock: Clock.tick(self)

In [92]:
class CalendarClock(Clock, Calendar):
    """ 
        The class CalendarClock implements a clock with integrated 
        calendar. It's a case of multiple inheritance, as it inherits 
        both from Clock and Calendar      
    """

    def __init__(self, day, month, year, hour, minute, second):
        Clock.__init__(self,hour, minute, second)
        Calendar.__init__(self, day, month, year)


    def tick(self):
        """
        advance the clock by one second
        """
        previous_hour = self._hours
        Clock.tick(self)
        if (self._hours < previous_hour): 
            self.advance()

    def __str__(self):
        return Calendar.__str__(self) + ", " + Clock.__str__(self)


if __name__ == "__main__":
    x = CalendarClock(31, 12, 2013, 23, 59, 59)
    print("One tick from ",x, end=" ")
    x.tick()
    print("to ", x)

    x = CalendarClock(28, 2, 1900, 23, 59, 59)
    print("One tick from ",x, end=" ")
    x.tick()
    print("to ", x)

    x = CalendarClock(28, 2, 2000, 23, 59, 59)
    print("One tick from ",x, end=" ")
    x.tick()
    print("to ", x)

    x = CalendarClock(7, 2, 2013, 13, 55, 40)
    print("One tick from ",x, end=" ")
    x.tick()
    print("to ", x)

One tick from  12/31/2013, 23:59:59 to  01/01/2014, 00:00:00
One tick from  02/28/1900, 23:59:59 to  03/01/1900, 00:00:00
One tick from  02/28/2000, 23:59:59 to  02/29/2000, 00:00:00
One tick from  02/07/2013, 13:55:40 to  02/07/2013, 13:55:41


#### The Diamond Problem or the 'deadly diamond of death'

**The "diamond problem" (sometimes referred as the "deadly diamond of death") is the generally used term for an ambiguity that arises when two classes B and C inherit from a superclass A, and another class D inherits from both B and C. If there is a method "m" in A that B or C (or even both of them) has overridden, and furthermore, if it does not override this method, then the question is which version of the method does D inherit? It could be the one from A, B or C**

Let's look at Python. The first Diamond Problem configuration is like this: Both B and C override the method m of A:

In [98]:
class A:
    def m(self):
        print("m of A called")

class B(A):
    def m(self):
        print("m of B called")
    
class C(A):
    def m(self):
        print("m of C called")

class D(C, B):
    pass

In [97]:
x = D()
x.m()

m of B called


If you call the method m on an instance x of D, i.e. x.m(), we will get the output "m of B called". If we transpose the order of the classes in the class header of D in "class D(C,B):", we will get the output "m of C called".

The case in which m will be overridden only in one of the classes B or C, e.g. in C:

In [99]:
class A:
    def m(self):
        print("m of A called")

class B(A):
    pass
    
class C(A):
    def m(self):
        print("m of C called")

class D(B,C):
    pass

x = D()
x.m()

m of C called



Principially, two possibilities are imaginable: "m of C" or "m of A" could be used

We call this script with Python2.7 (python) and with Python3 (python3) to see what's happening:

```
$ python diamond1.py 
m of A called
$ python3 diamond1.py 
m of C called
```

Only for those who are interested in Python version2: To have the same inheritance behaviour in Python2 as in Python3, every class has to inherit from the class "object". Our class A doesn't inherit from object, so we get a so-called old-style class, if we call the script with python2. Multiple inheritance with old-style classes is governed by two rules: depth-first and then left-to-right. If you change the header line of A into "class A(object):", we will have the same behaviour in both Python versions.

#### super and MRO

We have seen in our previous implementation of the diamond problem, how Python "solves" the problem, i.e. in which order the base classes are browsed through. The order is defined by the so-called "Method Resolution Order" or in short MRO.

We will extend our previous example, so that every class defines its own method m:

In [100]:
class A:
    def m(self):
        print("m of A called")

class B(A):
    def m(self):
        print("m of B called")
    
class C(A):
    def m(self):
        print("m of C called")

class D(B,C):
    def m(self):
        print("m of D called")

Let's apply the method m on an instance of D. We can see that only the code of the method m of D will be executed. We can also explicitly call the methods m of the other classes via the class name, as we demonstrate in the following interactive Python session:

In [101]:
x = D()
B.m(x)

m of B called


In [102]:
C.m(x)

m of C called


In [103]:
A.m(x)

m of A called


In [105]:
x.m()

m of D called


Now let's assume that the method m of D should execute the code of m of B, C and A as well, when it is called. We could implement it like this:

In [106]:
class D(B,C):
    def m(self):
        print("m of D called")
        B.m(self)
        C.m(self)
        A.m(self)

In [107]:
# The output is what we have been looking for:

x = D()
x.m()

m of D called
m of B called
m of C called
m of A called


But it turns out once more that things are more complicated than they seem. How can we cope with the situation, if both m of B and m of C will have to call m of A as well. In this case, we have to take away the call A.m(self) from m in D. The code might look like this, but there is still a bug lurking in it:

In [108]:

class A:
    def m(self):
        print("m of A called")

class B(A):
    def m(self):
        print("m of B called")
        A.m(self)
    
class C(A):
    def m(self):
        print("m of C called")
        A.m(self)

class D(B,C):
    def m(self):
        print("m of D called")
        B.m(self)
        C.m(self)

The bug is that the method m of A will be called twice:

In [109]:
x = D()
x.m()

m of D called
m of B called
m of A called
m of C called
m of A called


One way to solve this problem - admittedly not a Pythonic one - consists in splitting the methods m of B and C in two methods. The first method, called _m consists of the specific code for B and C and the other method is still called m, but consists now of a call self._m() and a call A.m(self). The code of the method m of D consists now of the specific code of D 'print("m of D called")', and the calls B._m(self), C._m(self) and A.m(self):

In [110]:
class A:
    def m(self):
        print("m of A called")

class B(A):
    def _m(self):
        print("m of B called")
    def m(self):
        self._m()
        A.m(self)
    
class C(A):
    def _m(self):
        print("m of C called")
    def m(self):
        self._m()
        A.m(self)

class D(B,C):
    def m(self):
        print("m of D called")
        B._m(self)
        C._m(self)
        A.m(self)

Our problem is solved, but - as we have already mentioned - not in a pythonic way:

In [111]:
x = D()
x.m()

m of D called
m of B called
m of C called
m of A called


The optimal way to solve the problem, which is the "super" pythonic way, would be calling the super function:

In [112]:
class A:
    def m(self):
        print("m of A called")

class B(A):
    def m(self):
        print("m of B called")
        super().m()
    
class C(A):
    def m(self):
        print("m of C called")
        super().m()

class D(B,C):
    def m(self):
        print("m of D called")
        super().m()

It also solves our problem, but in a beautiful design as well:

In [113]:
x = D()
x.m()

m of D called
m of B called
m of C called
m of A called


The super function is often used when instances are initialized with the \_\_init__ method:

In [125]:
class A:
    m = 'a'
    def __init__(self):
        print("A.__init__")

class B(A):
    m = 'b'
    def __init__(self):
        print("B.__init__")
        super().__init__()
    
class C(A):
    m = 'c'
    def __init__(self):
        print("C.__init__")
        super().__init__()


class D(B,C):
    def __init__(self):
        print("D.__init__")
        super().__init__()

We demonstrate the way of working in the following interactive session:

In [115]:
d = D()

D.__init__
B.__init__
C.__init__
A.__init__


In [116]:
c = C()

C.__init__
A.__init__


In [117]:
b = B()

B.__init__
A.__init__


In [118]:
a = A()

A.__init__


The question arises about how the super functions makes decisions. How does it decide which class has to be used? As we have already mentioned, it uses the so-called method resolution order(MRO). It is based on the "[C3 superclass linearisation](https://www.youtube.com/watch?v=cuonAMJjHow)" algorithm (used since python 2.3). This is called a linearisation, because the tree structure is broken down into a linear order. The mro method can be used to create this list:

In [119]:
D.mro()

[__main__.D, __main__.B, __main__.C, __main__.A, object]

In [120]:
B.mro()

[__main__.B, __main__.A, object]

In [121]:
A.mro()

[__main__.A, object]

In [126]:
x = D()
x.m

D.__init__
B.__init__
C.__init__
A.__init__


'b'

#### Polymorphism

Polymorphism in Computer Science is the ability to present the same interface for differing underlying forms. We can have in some programming languages polymorphic functions or methods, for example. Polymorphic functions or methods can be applied to arguments of different types, and they can behave differently depending on the type of the arguments to which they are applied. We can also define the same function name with a varying number of parameter.

Let's have a look at the following Python function:

In [127]:
def f(x, y):
    print("values: ", x, y)

f(42, 43)
f(42, 43.7) 
f(42.3, 43)
f(42.0, 43.9)

values:  42 43
values:  42 43.7
values:  42.3 43
values:  42.0 43.9


We can call this function with various types, as demonstrated in the example. In typed programming languages like Java or C++, we would have to overload f to implement the various type combinations.

Python is implicitly polymorphic. We can apply our previously defined function f even to lists, strings or other types, which can be printed:

In [128]:
f([3,5,6],(3,5))

values:  [3, 5, 6] (3, 5)


## Magic methods and operator overloading

![](https://www.python-course.eu/images/marvin_the_magician_400w.webp)

The so-called magic methods have nothing to do with wizardry. You have already seen them in the previous chapters of our tutorial. They are special methods with fixed names. They are the methods with this clumsy syntax, i.e. the double underscores at the beginning and the end. They are also hard to talk about. How do you pronounce or say a method name like \_\_init__? "Underscore underscore init underscore underscore" sounds horrible and is almost a tongue twister. "Double underscore init double underscore" is a lot better, but the ideal way is "dunder init dunder" That's why magic methods methods are sometimes called dunder methods!

So what's magic about the \_\_init__ method? The answer is, you don't have to invoke it directly. The invocation is realized behind the scenes. When you create an instance x of a class A with the statement "x = A()", Python will do the necessary calls to [\_\_new__](https://howto.lintel.in/python-__new__-magic-method-explained/) and \_\_init__

We have encountered the concept of operator overloading many times in the course of this tutorial. We had used the plus sign to add numerical values, to concatenate strings or to combine lists:

In [129]:
4 + 5

9

In [130]:
3.8 + 9

12.8

In [131]:
"Peter" + " " + "Pan"

'Peter Pan'

In [132]:
[3,6,8] + [7,11,13]

[3, 6, 8, 7, 11, 13]

Operator Overloading \_\_add__
It's even possible to overload the "+" operator as well as all the other operators for the purposes of your own class. To do this, you need to understand the underlying mechanism. There is a special (or a "magic") method for every operator sign. The magic method for the "+" sign is the \_\_add__ method. For "-" it is \_\_sub__ and so on. We have a complete listing of all the magic methods a little further down.

The mechanism works like this: If we have an expression "x + y" and x is an instance of class K, then Python will check the class definition of K. If K has a method \_\_add__ it will be called with x.\_\_add__(y), otherwise we will get an error message:

```
Traceback (most recent call last):
  File "", line 1, in 
TypeError: unsupported operand type(s) for +: 'K' and 'K'
```

#### Overview of Magic Methods

Binary Operators
```
Operator	Method

+	object.__add__(self, other)
-	object.__sub__(self, other)
*	object.__mul__(self, other)
//	object.__floordiv__(self, other)
/	object.__truediv__(self, other)
%	object.__mod__(self, other)
**	object.__pow__(self, other[, modulo])
<<	object.__lshift__(self, other)
>>	object.__rshift__(self, other)
&	object.__and__(self, other)
^	object.__xor__(self, other)
|	object.__or__(self, other)
```

Extended Assignments
```
Operator	Method
+=	object.__iadd__(self, other)
-=	object.__isub__(self, other)
*=	object.__imul__(self, other)
/=	object.__idiv__(self, other)
//=	object.__ifloordiv__(self, other)
%=	object.__imod__(self, other)
**=	object.__ipow__(self, other[, modulo])
<<=	object.__ilshift__(self, other)
>>=	object.__irshift__(self, other)
&=	object.__iand__(self, other)
^=	object.__ixor__(self, other)
|=	object.__ior__(self, other)
```

Unary Operators
```
Operator	Method
-	object.__neg__(self)
+	object.__pos__(self)
abs()	object.__abs__(self)
~	object.__invert__(self)
complex()	object.__complex__(self)
int()	object.__int__(self)
long()	object.__long__(self)
float()	object.__float__(self)
oct()	object.__oct__(self)
hex()	object.__hex__(self)
```

Comparison Operators
```
Operator	Method
<	object.__lt__(self, other)
<=	object.__le__(self, other)
==	object.__eq__(self, other)
!=	object.__ne__(self, other)
>=	object.__ge__(self, other)
>	object.__gt__(self, other)
```

#### Example class: Length

We will demonstrate the Length class and how you can overload the "+" operator for your own class. To do this, we have to overload the \_\_add__ method. Our class contains the \_\_str__ and \_\_repr__ methods as well. The instances of the class Length contain length or distance information. The attributes of an instance are self.value and self.unit.

This class allows us to calculate expressions with mixed units like this one:

```2.56 m + 3 yd + 7.8 in + 7.03 cm```

In [133]:
class Length:

    __metric = {"mm" : 0.001, "cm" : 0.01, "m" : 1, "km" : 1000,
                "in" : 0.0254, "ft" : 0.3048, "yd" : 0.9144,
                "mi" : 1609.344 }
    
    def __init__(self, value, unit = "m" ):
        self.value = value
        self.unit = unit
    
    def Converse2Metres(self):
        return self.value * Length.__metric[self.unit]
    
    def __add__(self, other):
        l = self.Converse2Metres() + other.Converse2Metres()
        return Length(l / Length.__metric[self.unit], self.unit )

    # We use the method __iadd__ to implement the extended assignment:
    def __iadd__(self, other):
        if type(other) == int or type(other) == float:
            l = self.Converse2Metres() + other
        else:
            l = self.Converse2Metres() + other.Converse2Metres()
        self.value = l / Length.__metric[self.unit]
        return self
    # Now we are capable of writing the following assignments:
    # x += Length(1)
    # x += Length(4, "yd")
    
    def __str__(self):
        return str(self.Converse2Metres())
    
    def __repr__(self):
        return "Length(" + str(self.value) + ", '" + self.unit + "')"

if __name__ == "__main__":
    x = Length(4)
    print(x)
    y = eval(repr(x))

    z = Length(4.5, "yd") + Length(1)
    print(repr(z))
    print(z)

4
Length(5.593613298337708, 'yd')
5.1148


It's a safe bet that if somebody works with adding integers and floats from the right side for a while, he or she will want to have the same from the left side! SWhat will happen, if we execute the following code line:

In [134]:
x = 5 + Length(3, "yd")

TypeError: ignored

Of course, the left side has to be of type "Length", because otherwise Python tries to apply the \_\_add__ method from int, which can't cope with Length objects as second arguments!

Python provides a solution for this problem as well. It's the \_\_radd__ method. It works like this: Python tries to evaluate the expression "5 + Length(3, 'yd')". First it calls int.\_\_add__(5,Length(3, 'yd')), which will raise an exception. After this it will try to invoke Length.\_\_radd__(Length(3, "yd"), 5). It's easy to recognize that the implementation of \_\_radd__ is analogue to \_\_add__:

In [136]:
class Length:

    __metric = {"mm" : 0.001, "cm" : 0.01, "m" : 1, "km" : 1000,
                "in" : 0.0254, "ft" : 0.3048, "yd" : 0.9144,
                "mi" : 1609.344 }
    
    def __init__(self, value, unit = "m" ):
        self.value = value
        self.unit = unit
    
    def Converse2Metres(self):
        return self.value * Length.__metric[self.unit]
    
    def __add__(self, other):
        l = self.Converse2Metres() + other.Converse2Metres()
        return Length(l / Length.__metric[self.unit], self.unit )

    # We use the method __iadd__ to implement the extended assignment:
    def __iadd__(self, other):
        if type(other) == int or type(other) == float:
            l = self.Converse2Metres() + other
        else:
            l = self.Converse2Metres() + other.Converse2Metres()
        self.value = l / Length.__metric[self.unit]
        return self
    # Now we are capable of writing the following assignments:
    # x += Length(1)
    # x += Length(4, "yd")

    def __radd__(self, other):
        if type(other) == int or type(other) == float:
            l = self.Converse2Metres() + other
        else:
            l = self.Converse2Metres() + other.Converse2Metres()
        return Length(l / Length.__metric[self.unit], self.unit )

    def __str__(self):
        return str(self.Converse2Metres())
    
    def __repr__(self):
        return "Length(" + str(self.value) + ", '" + self.unit + "')"

if __name__ == "__main__":
    x = Length(4)
    print(x)
    y = eval(repr(x))

    z = Length(4.5, "yd") + Length(1)
    print(repr(z))
    print(z)

4
Length(5.593613298337708, 'yd')
5.1148


In [138]:
x = 5 + Length(3, "yd")
x

Length(8.468066491688539, 'yd')

This means that all the previously introduced binary and extended assignment operators exist in the "reversed" version as well:

```
__radd__ , __rsub__ , __rmul__ etc.
```

#### Standard Classes as Base Classes

It's possible to use standard classes - like int, float, dict or lists - as base classes as well.

We extend the list class by adding a push method:

In [139]:
class Plist(list):

    def __init__(self, l):
        list.__init__(self, l)

    def push(self, item):
        self.append(item)


if __name__ == "__main__":
    x = Plist([3,4])
    x.push(47)
    print(x)

[3, 4, 47]
